In [32]:
input_string = '(a+C*a+a)<=C'

In [33]:
import json

In [34]:
first = dict()
first['exp'] = ['a', 'C', '(']
first['arith_exp'] = ['a', 'C', '(']
first['arith_exp_dash'] = ['+', '-', 'e']
first['term'] = ['a', 'C', '(']
first['term_dash'] = ['*', '/', 'e']
first['factor'] = ['a', 'C', '(']
first['ident'] = ['a']
first['const'] = ['C']
first['rel'] = ['<', '=', '>']
first['plus'] = ['+', '-']
first['mul'] = ['*', '/']
first['operator'] = ['a']

In [35]:
def get_first(nonterm):
    global i
    if i > len(input_string) - 1:
        return False
    is_nonterm = input_string[i] in first[nonterm]
    return is_nonterm

def rel():
    global tree
    tree += '{"Rel":['
    global i
    if input_string[i] == '<':
        tree += '"'
        tree += input_string[i]
        tree += '"'
        i += 1
        if input_string[i] == '=':
            tree += ','
            tree += '"'
            tree += input_string[i]
            tree += '"'
            i += 1
        elif input_string[i] == '>':
            tree += ','
            tree += '"'
            tree += input_string[i]
            tree += '"'
            i += 1
    elif input_string[i] == '=':
        tree += '"'
        tree += input_string[i]
        tree += '"'
        i += 1
    elif input_string[i] == '>':
        tree += '"'
        tree += input_string[i]
        tree += '"'
        i += 1
        if input_string[i] == '=':
            tree += ','
            tree += '"'
            tree += input_string[i]
            tree += '"'
            i += 1
    else:
        raise Exception('Unexpected symbol on ' + str(i))
    tree += ']}'

def const():
    global tree
    tree += '{"Const":['
    global i
    if get_first('const'):
        tree += '"'
        tree += input_string[i]
        tree += '"'
        i += 1
    else:
        raise Exception('Unexpected symbol on ' + str(i))
    tree += ']}'
    
def ident():
    global tree
    tree += '{"Id":['
    global i
    if get_first('ident'):
        tree += '"'
        tree += input_string[i]
        tree += '"'
        i += 1
    else:
        raise Exception('Unexpected symbol on ' + str(i))
    tree += ']}'

def mul():
    global tree
    tree += '{"Mul":['
    global i
    if get_first('mul'):
        tree += '"'
        tree += input_string[i]
        tree += '"'
        i += 1
    else:
        raise Exception('Unexpected symbol on ' + str(i))
    tree += ']}'

def plus():
    global tree
    tree += '{"Plus":['
    global i
    if get_first('plus'):
        tree += '"'
        tree += input_string[i]
        tree += '"'
        i += 1
    else:
        raise Exception('Unexpected symbol on ' + str(i))
    tree += ']}'

def factor():
    global tree
    tree += '{"Factor":['
    global i
    if get_first('ident'):
        ident()
    elif get_first('const'):
        const()
    elif input_string[i] == '(':
        tree += '"(",'
        i += 1
        exp()
        tree += ','
        if input_string[i] != ')':
            raise Exception('Mismatching brackets')
        else:
            i += 1
            tree += '")"'
    else:
        raise Exception('Unexpected symbol on ' + str(i))
    tree += ']}'

def term_dash():
    global tree
    tree += '{"Term_Dash":['
    if get_first('mul'):
        mul()
        tree += ','
        factor()
        tree += ','
        term_dash()
    tree += ']}'

def term():
    global tree
    tree += '{"Term":['
    factor()
    tree += ','
    term_dash()
    tree += ']}'

def arith_exp_dash():
    global tree
    tree += '{"Arithmetic_Expression_Dash":['
    if get_first('plus'):
        plus()
        tree += ','
        term()
        tree += ','
        arith_exp_dash()
    tree += ']}'

def arith_exp():
    global tree
    tree += '{"Arithmetic_Expression":['
    term()
    tree += ','
    arith_exp_dash()
    tree += ']}'

def exp():
    global tree
    tree += '{"Expression":['
    if get_first('arith_exp'):
        arith_exp()
        if get_first('rel'):
            tree += ','
            rel()
            tree += ','
            arith_exp() 
    else:
        raise Exception('Unexpected symbol on ' + str(i))
    tree += ']}'   

def parse():
    global i 
    global tree

    i = 0
    tree = ''

    try:
        exp()
        if i < len(input_string):
           raise Exception('Unexpected symbol on ' + str(i))
        print('Ok!')
        return True
    except Exception as e: 
        print(e)
        return False

i = 0
tree = ''
result = dict()
if parse():
    result = json.loads(tree)

Ok!


In [36]:
result

{'Expression': [{'Arithmetic_Expression': [{'Term': [{'Factor': ['(',
        {'Expression': [{'Arithmetic_Expression': [{'Term': [{'Factor': [{'Id': ['a']}]},
              {'Term_Dash': []}]},
            {'Arithmetic_Expression_Dash': [{'Plus': ['+']},
              {'Term': [{'Factor': [{'Const': ['C']}]},
                {'Term_Dash': [{'Mul': ['*']},
                  {'Factor': [{'Id': ['a']}]},
                  {'Term_Dash': []}]}]},
              {'Arithmetic_Expression_Dash': [{'Plus': ['+']},
                {'Term': [{'Factor': [{'Id': ['a']}]}, {'Term_Dash': []}]},
                {'Arithmetic_Expression_Dash': []}]}]}]}]},
        ')']},
      {'Term_Dash': []}]},
    {'Arithmetic_Expression_Dash': []}]},
  {'Rel': ['<', '=']},
  {'Arithmetic_Expression': [{'Term': [{'Factor': [{'Const': ['C']}]},
      {'Term_Dash': []}]},
    {'Arithmetic_Expression_Dash': []}]}]}

In [37]:
def parse_exp(elem):
    elem = elem['Expression']
    if len(elem) > 1:
        parse_arith(elem[0])
        parse_arith(elem[2])
        parse_rel(elem[1])
    else:
        parse_arith(elem[0])

def parse_arith(elem):
    elem = elem['Arithmetic_Expression']
    parse_term(elem[0])
    parse_arith_dash(elem[1])

def parse_arith_dash(elem):
    elem = elem['Arithmetic_Expression_Dash']
    if len(elem) > 0:
        parse_term(elem[1])
        parse_arith_dash(elem[2])
        parse_plus(elem[0])

def parse_term(elem):
    elem = elem['Term']
    parse_factor(elem[0])
    parse_term_dash(elem[1])

def parse_term_dash(elem):
    elem = elem['Term_Dash']
    if len(elem) > 0:
        parse_factor(elem[1])
        parse_term_dash(elem[2])
        parse_mul(elem[0])

def parse_factor(elem):
    global backward
    elem = elem['Factor']
    if len(elem) > 1:
        parse_exp(elem[1])
    else:
        backward += list(elem[0].values())[0][0]
        backward += ' '

def parse_rel(elem):
    global backward
    elem = elem['Rel']
    backward += ''.join(elem)
    backward += ' '

def parse_plus(elem):
    global backward
    elem = elem['Plus']
    backward += elem[0]
    backward += ' '

def parse_mul(elem):
    global backward
    elem = elem['Mul']
    backward += elem[0]
    backward += ' '

backward = ''
parse_exp(result)
print(backward)


a C a * a + + C <= 


In [39]:
input_string

'(a+C*a+a)<=C'